In [3]:
import os
os.getcwd()

'/content'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
dir = os.path.join('drive','My Drive','Colab Notebooks','STAT628_module3')
dir

'drive/My Drive/Colab Notebooks/STAT628_module3'

In [0]:
import numpy as np
import pandas as pd
import spacy
import re  ## regexp

In [0]:
# merge = pd.read_csv('dataset/merged.csv')
# merge.shape

In [22]:
merge = pd.read_csv(os.path.join(dir,'dataset','merged.csv'))
merge.shape

(123397, 22)

In [0]:
merge['text'] = merge.apply(lambda x: x['text'].lower(), axis=1)

In [0]:
def reg(x):
    sub = re.sub("ice cream","icecream",x)  ## addible
    return sub

In [0]:
merge['text'] = merge['text'].apply(reg)

In [0]:
#merge.head(2)

#### remove stopwords

In [0]:
# import nltk
# #nltk.download('stopwords')
# from nltk.corpus import stopwords

In [0]:
#print(set(stopwords.words('english')))
nlp = spacy.load("en")
nlp.Defaults.stop_words |= {".","'","[","]","(",")"}

In [0]:
print(nlp.Defaults.stop_words)

{'although', 'he', 'hence', 'quite', 'some', 'n‘t', 'sixty', 'thereupon', 'show', 'back', 'ten', 'under', 'yet', 'first', 'noone', 'whither', 'either', '’ve', 'fifty', 'anywhere', 'becoming', 'always', 'if', 'thence', 'enough', 'there', 'your', 'herself', 'via', 'not', 'every', 'two', 'they', 'amount', 'forty', 'part', 'it', 'more', 'formerly', 'myself', 're', 'them', 'among', 'most', 'before', 'beyond', 'had', 'seem', 'nobody', 'wherever', 'whose', 'their', '‘s', 'much', 'ourselves', 'five', '’d', 'since', "'ll", '’re', 'upon', 'does', 'once', 'our', 'this', 'then', 'hereby', 'into', 'whether', 'also', 'say', 'less', 'besides', 'least', 'make', 'somehow', 'everyone', 'six', 'so', 'might', '’ll', 'one', 'eleven', 'amongst', 'afterwards', 'herein', 'hundred', 'put', 'sometimes', '’s', 'being', '.', 'about', 'whereafter', 'and', "n't", 'thereby', 'full', 'mine', 'beside', 'go', 'used', 'onto', 'while', 'me', 'cannot', 'never', 'anything', 'move', 'themselves', 'other', 'serious', 'using'

In [0]:
def stop(sentence):
    #sub = re.sub("ice cream","icecream",x['text'])  ## addible
    res = list(filter(lambda x: False if x.is_stop else True, nlp(sentence)))
    return res

In [0]:
merge['text'] = merge['text'].apply(stop)

In [0]:
merge.head(2)

,Unnamed: 0,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,state,review_id,user_id,stars,useful,funny,cool,text,date
0,0,9020 B Albemarle Rd,"{'BusinessAcceptsCreditCards': 'True', 'Restau...",irft4YkdNsww4DNf_Aftew,"Juice Bars & Smoothies, Food, Ice Cream & Froz...",Charlotte,"{'Monday': '11:30-20:30', 'Tuesday': '11:30-20...",0,35.21051,-80.690262,So Cool Frozen Yogurt,28227,6,NC,zwHQZoAqsEtJ9IlQjS_hIQ,1WSTFGDksl3bh0TBW1AHCw,3.0,0,0,0,"[time, went, hair, icecream, broken, glass, gr...",2017-08-17 05:35:35
1,1,9020 B Albemarle Rd,"{'BusinessAcceptsCreditCards': 'True', 'Restau...",irft4YkdNsww4DNf_Aftew,"Juice Bars & Smoothies, Food, Ice Cream & Froz...",Charlotte,"{'Monday': '11:30-20:30', 'Tuesday': '11:30-20...",0,35.21051,-80.690262,So Cool Frozen Yogurt,28227,6,NC,Ktv2KlhExTOIkriWl6M8_Q,S88QtT4esAcmYGD0Icx4hw,5.0,1,0,0,"[pretty, good, froyo, place, toppings, bubble,...",2016-09-30 23:29:28


In [0]:
#merge.to_csv(os.path.join(dir,'dataset','merge_after_stop.csv'),index=False)

In [0]:
#merge2 = pd.read_csv(os.path.join(dir,'dataset','merge_after_stop.csv'))

In [0]:
#merge2.head(2)

,Unnamed: 0,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,state,review_id,user_id,stars,useful,funny,cool,text,date
0,0,9020 B Albemarle Rd,"{'BusinessAcceptsCreditCards': 'True', 'Restau...",irft4YkdNsww4DNf_Aftew,"Juice Bars & Smoothies, Food, Ice Cream & Froz...",Charlotte,"{'Monday': '11:30-20:30', 'Tuesday': '11:30-20...",0,35.21051,-80.690262,So Cool Frozen Yogurt,28227,6,NC,zwHQZoAqsEtJ9IlQjS_hIQ,1WSTFGDksl3bh0TBW1AHCw,3.0,0,0,0,"[time, went, hair, icecream, broken, glass, gr...",2017-08-17 05:35:35
1,1,9020 B Albemarle Rd,"{'BusinessAcceptsCreditCards': 'True', 'Restau...",irft4YkdNsww4DNf_Aftew,"Juice Bars & Smoothies, Food, Ice Cream & Froz...",Charlotte,"{'Monday': '11:30-20:30', 'Tuesday': '11:30-20...",0,35.21051,-80.690262,So Cool Frozen Yogurt,28227,6,NC,Ktv2KlhExTOIkriWl6M8_Q,S88QtT4esAcmYGD0Icx4hw,5.0,1,0,0,"[pretty, good, froyo, place, toppings, bubble,...",2016-09-30 23:29:28


#### change word form

In [0]:
import nltk
#nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
wnl = WordNetLemmatizer()

In [0]:
def retrive_text(sentence):
    sentence = list(map(lambda x: x.text,sentence))
    return sentence
def form_word(sentence):
    sentence = list(map(lambda x: wnl.lemmatize(x,'v'),sentence))
    sentence = list(map(lambda x: wnl.lemmatize(x,'n'),sentence))
    sentence = list(map(lambda x: wnl.lemmatize(x,'a'),sentence))
    return sentence

In [0]:
merge['text'] = merge['text'].apply(retrive_text)
merge['text'] = merge['text'].apply(form_word)

In [0]:
merge['text'][0]

['time',
 'go',
 'hair',
 'icecream',
 'break',
 'glass',
 'grind',
 'door.t',
 'best',
 'place',
 'eat',
 'want',
 'icecream',
 'guest']

In [0]:
type(merge['text'][0][0])

str

### word count

In [0]:
inte = pd.read_csv(os.path.join(dir,'dataset','inte.csv'))

In [0]:
import torch
from torchtext import data
from torchtext.data import TabularDataset
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',lower=True)
LABEL = data.LabelField(dtype = torch.float)

In [0]:
#inte = merge[['text','stars']]
#inte.head()

In [0]:
#inte.to_csv(os.path.join(dir,'dataset','inte.csv'),index=False)

In [0]:
txt_datafields = [("text", TEXT), # we won't be needing the id, so we pass in None as the field
                  ("stars", LABEL)]
txt = TabularDataset(
           path=os.path.join(dir,'dataset','inte.csv'), # the file path
           format='csv',
           skip_header=True, # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
           fields=txt_datafields)

In [14]:
print(f'Number of testing examples: {len(txt)}')

Number of testing examples: 123397


### build vocab

In [15]:
TEXT.build_vocab(txt)
LABEL.build_vocab(txt)
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 63581
Unique tokens in LABEL vocabulary: 5


In [16]:
common = TEXT.vocab.freqs.most_common(514)
total_count = pd.DataFrame(common,columns=['word','count'])
total_count.drop(index = [0,1,2,3,4,5,9,10,18,25,47,83,106,139],inplace = True)
total_count.head()

,word,count
6,icecream,95811
7,place,79895
8,good,67799
11,like,53533
12,flavor,53018


In [17]:
inte.shape

(123397, 2)

### word count in each review

In [0]:
df = pd.DataFrame()
for i in range(inte.shape[0]):
    ex0 = vars(txt.examples[i])['text']
    count = [ex0.count(i) for i in total_count['word']]
    pd0 = pd.DataFrame(count).transpose()
    pd0.columns = total_count['word']
    df = df.append(pd0)

In [0]:
list(filter(lambda x: False if x.is_stop else True, sentence))

[word, definitelynotastopword]

In [19]:
df

word,icecream,place,good,like,flavor,try,time,great,order,come,get,love,chocolate,go,service,taste,food,?,delicious,sweet,want,nice,$,friendly,little,best,think,yogurt,definitely,serve,gelato,look,staff,price,wait,eat,location,ice,know,day,...,onion,lol,sesame,bake,strip,soda,literally,head,problem,fro,+,honestly,spoon,set,healthy,list,smell,past,picture,bread,job,fine,pecan,mouth,meat,prefer,thai,concept,7,mochi,average,room,syrup,queen,sunday,self,play,velvet,son,write
0,2,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0,2,1,1,1,0,0,0,0,0,1,0,0,0,0,1,0,4,0,0,1,0,0,0,0,0,0,2,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,1,0,0,0,0,1,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,3,0,0,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,2,1,2,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
df.to_csv(os.path.join(dir,'dataset','count.csv'),index=False)

In [0]:
word_count_label = pd.concat([df,merge],axis = 1)

In [0]:
word_count_label.to_csv(os.path.join(dir,'dataset','word_count_label.csv'),index=False)